In [ ]:
import nltk

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [3]:
raw = [
    'Run a marathon in under two hours. Impossible? Not for Nike (@Nike). Last May, the company brought three of the best runners on the planet together in Italy to set a new record in a closed-door marathon that was broadcast live on Twitter.'
    ,
    'For Pandora (@pandoramusic), there is bound to be a piece of music for each emoji. At least this is what the American music streaming giant wanted to prove with its campaign #SoundsLike.']
import numpy as np
import pandas as pd

df = pd.DataFrame(raw)
df.head()

,0
0,Run a marathon in under two hours. Impossible?...
1,"For Pandora (@pandoramusic), there is bound to..."


In [4]:
tag_list = ['DeepstateVirus', 'DeepStateVaccine', 'DeepStateFauci', 'QAnon', 'Agenda21', 'CCPVirus',
            'ClimateChangeHoax', 'GlobalWarmingHoax', 'ChinaLiedPeopleDied', 'SorosVirus', '5GCoronavirus', 'MAGA',
            'WWG1WGA', 'Chemtrails', 'flatEarth', 'MoonLandingHoax', 'moonhoax', 'illuminati', 'pizzaGateIsReal',
            'PedoGateIsReal', '911truth', '911insidejob', 'reptilians']

tag_list_lower = list(map(lambda x: x.lower(), tag_list))



In [8]:
sid.polarity_scores(df[0][0])
# df['scores'] = df[0].apply(lambda review: sid.polarity_scores(review))

# df.head()

{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'compound': 0.6369}

In [6]:
from psycopg2.extras import RealDictCursor
import psycopg2

""" Connect to the PostgreSQL database server """
conn = None


def exec_sql(sql: str):
    try:
        conn = psycopg2.connect("dbname=data user=postgres password=root host=172.18.0.2")

        # create a cursor
        cur = conn.cursor(cursor_factory=RealDictCursor)
        # cur = conn.cursor()

        # execute a statement
        print('PostgreSQL database version:')
        cur.execute(sql)

        # display the PostgreSQL database server version
        db_version = cur.fetchall()

        # close the communication with the PostgreSQL
        cur.close()
        return db_version
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


In [ ]:
# for i in tag_list_lower:
#     sql = INSERT INTO public.hoaxs (id, name) VALUES (null, 'aaa')

In [19]:
import re

sql = f'select * from hashtags where LOWER(value) in {str(tuple(tag_list_lower))}'
rows = exec_sql(sql)
tag_list_ids = list(map(lambda x: x['id'], rows))
# print(tag_list_ids)


sql = f'select tweet_id from  tweet_hashtags where hashtag_id in {str(tuple(tag_list_ids))} limit 500'
rows = exec_sql(sql)

for row in rows:
    processing_id = row['tweet_id']
    sql = f'select content from tweets where id = \'{processing_id}\''
    print(exec_sql(sql))
def filter_content(content: str):
    # article = re.sub(r'[]', '</html>', article)
    pass




PostgreSQL database version:
Database connection closed.
PostgreSQL database version:
Database connection closed.
PostgreSQL database version:
Database connection closed.
[RealDictRow([('content', '#USA and #India, Wary of @china, Agree to Strengthen Military Ties !!\n\n#TradeWar\n#MIGA\n#MAGA\n#innovation\n\n@nytimes @cnnbrk @WSJ\n@VP @realDonaldTrump @XHNews\n@AlJazeera_World @CCTV @avigyanSingh18 @ChinaDailyUSA @ChinaPlusNews\nhttps://t.co/Bo5j98cc6g')])]
PostgreSQL database version:
Database connection closed.
[RealDictRow([('content', 'Secretly delivering American technology\nto China since 1995 - Dianne Feinstein\n#democrats #Apple #MAGA #California https://t.co/8v2GuXW4cF')])]
PostgreSQL database version:
Database connection closed.
[RealDictRow([('content', '.Hollywood including Ron Howard in bed w/ China: Chinese investors flood N2 Hollywood https://t.co/OwzeoIxhRH via @financialtimes\n#Imagine #QAnon #MAGA')])]
PostgreSQL database version:
Database connection closed.
[RealDic